In [1]:
import torch
from torch.autograd import Variable # torch의 automatic gradient calculate하는 부분에서 variable을 가지고 옴.

In [3]:
x_tensor=torch.Tensor(3,4)
x_tensor #x_tensor.data 사용할 수 없음.

tensor([[0.0000e+00, 0.0000e+00, 1.8264e-37, 1.4013e-45],
        [1.4013e-45, 0.0000e+00,        nan,        nan],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 2.7551e-40]])

In [5]:
x_variable= Variable(x_tensor) #variable로 감싸주면 값은 똑같아 보이지만 사실 variable이라는 class로 감싸져 있는 것임. 
x_variable

tensor([[0.0000e+00, 0.0000e+00, 1.8264e-37, 1.4013e-45],
        [1.4013e-45, 0.0000e+00,        nan,        nan],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 2.7551e-40]])

In [6]:
#.data -> wrapped tensor
x_variable.data #tensor값들이 들어가게 됨.

tensor([[0.0000e+00, 0.0000e+00, 1.8264e-37, 1.4013e-45],
        [1.4013e-45, 0.0000e+00,        nan,        nan],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 2.7551e-40]])

In [8]:
#.grad ->gradient of the variable
print(x_variable.grad) # gradient값도 저장이됨. 아직 연산이 진행되지 않았기 때문에 None

None


In [10]:
#.requires_grad -> whether variable requires gradient. gradient를 계산할 지 말지의 여부가 들어가 있는 것임.
print(x_variable.requires_grad)

x_variable=Variable(x_tensor,requires_grad=True) #처음에 만들 때 gradient를 계산할 지 말지 정함.
x_variable.requires_grad

False


True

In [12]:
#.volatile -> inference mode with minimal memory usage #medel를 학습하고 테스트할 때 inference 할 때 많이 쓰는 것. 
#불필요한 연산 제거하고 앞으로 feed forward만 하게 최적화시키는 것
#전체 연상 graph에 volatile이 하나라도 있으면 전체가 inference mode로 바뀜
#single volatile variable makes whole graph not requiring gradient -> requires_grad도 자동 false로 바뀜.
#테스트할 때 값을 빨리빨리 확인해 볼 수 있음.

x_variable=Variable(x_tensor,volatile=True)
x_variable.grad, x_variable.requires_grad, x_variable.volatile

/Users/imyeonghui/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys
/Users/imyeonghui/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: volatile was removed (Variable.volatile is always False)
  


(None, False, False)

In [18]:
#모든 gradient 계산 여부가 True로 나옴. 
#create graph

x=Variable(torch.FloatTensor(3,4),requires_grad=True)
y=x**2+4*x
z=2*y+3

x.requires_grad, y.requires_grad, z.requires_grad ,x , y, z

(True,
 True,
 True,
 tensor([[ 0.0000e+00, -2.5244e-29,  6.7841e+27,  2.5250e-29],
         [ 4.2039e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  2.7551e-40]],
        requires_grad=True),
 tensor([[ 0.0000e+00, -1.0097e-28,         inf,  1.0100e-28],
         [ 1.6816e-44,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.1020e-39]],
        grad_fn=<AddBackward0>),
 tensor([[3., 3., inf, 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.]], grad_fn=<AddBackward0>))

In [19]:
#.backward(gradient,retain_graph,create_graph,retain_variables) #backpropagation을 하는 것임. 
#compute gradient of current variable w.r.t. graph leaves

loss=torch.FloatTensor(3,4)#임의의 loss값을 지정
z.backward(loss)#위에 만든 graph에

print(x.grad)
y.grad, z.grad #None으로 뜨는 이유는 위의 식이 모두 x로 표시될 수 있기 때문임. 모든 graph를 지나가지만, 값이 저장되는 것이 leaf noded인 x에만 저장이 됨.


tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


(None, None)